In [3]:
from scipy.optimize         import newton
from sklearn.neighbors      import KDTree
from scipy.sparse           import csr_matrix
from scipy.sparse.csgraph   import dijkstra
from scipy.linalg           import eig
from sklearn.decomposition  import PCA
import numpy                as np
import gudhi                as gd
import warnings
import gurobipy             as gp
from gurobipy               import GRB
import matplotlib.pyplot    as plt
from kneed                  import KneeLocator
from scipy.spatial          import distance_matrix
from ripser                 import Rips
from sklearn.preprocessing  import PolynomialFeatures
import sympy                as sp
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.problem     import Problem
from pymoo.optimize         import minimize

In [4]:
np.random.seed(1)

A = np.random.rand(4, 2)
y = np.random.randn(4)

q = np.random.rand(5)
c_1 = np.random.randn(5)

alpha = 1

b_prime = np.random.rand(2)
c_1_prime = np.random.rand(2)

In [8]:
m = gp.Model()
m.setParam('OutputFlag', 0)
m.setParam(GRB.Param.NonConvex, 2)
x = m.addMVar(shape=2, lb=float('-inf'))
z = m.addMVar(shape=2, lb=float('-inf'))

Q = A.T @ A
c = -2 * y.T @ A
obj = x @ Q @ x + c @ x + y.T @ y

m.setObjective(obj, GRB.MINIMIZE)

c0 = m.addConstr(x@x == alpha**2, name="c0")

c1 = m.addConstr(z == x+b_prime-c_1_prime, name="d0")
d1 = m.addConstr(z@z == (q-c_1)@(q-c_1), name="d0")

m.feasRelax(relaxobjtype=1, minrelax=False, vars=None, lbpen=None, ubpen=None, constrs=[d1], rhspen=[1])
m.optimize()

In [14]:
X = np.asarray(x.X)

In [15]:
print(X@X)
print((q-c_1)@(q-c_1))
print((X+b_prime-c_1_prime)@(X+b_prime-c_1_prime))

1.0000013485485761
4.13061337865229
3.2506623805725794


In [9]:
m

<gurobi.Model MIP instance Unnamed: 2 constrs, 6 vars, Parameter changes: NonConvex=2, OutputFlag=0>

In [10]:
m.update()
m.getQConstrs()

[<gurobi.QConstr c0>, <gurobi.QConstr d0>]

In [11]:
d1

<gurobi.QConstr d0>

In [24]:
m.feasRelax(1, False, m.getVars(), [1, 1], [1, 1], m.getConstrs(), [1, 1])

GurobiError: Length of penalty list differs from length of constraint list